In [ ]:
import matplotlib.pyplot as mpl
import matplotlib.patches as patches
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
from shapely.geometry import Point, Polygon, LineString
from descartes import PolygonPatch
import random
from collections import OrderedDict,Counter
from math import floor

In [ ]:
def random_color():
    r = lambda: random.randint(0,255)
    return '#%02X%02X%02X' % (r(),r(),r())


def to_point(pedestrian):
    return Point(pedestrian[2] / 1000, pedestrian[3] / -1000)


def to_path(points):
    return 


class Pedestrian:
    def __init__(self, frame):
        self.frame = frame
        
        self.frame[['x','y']] = self.frame[['x','y']] / 1000
        self.frame['y'] *=  -1
        
        self.id = self.frame.pedestrianId[0]
        self.start = Point(self.frame.x[0], self.frame.y[0])
        
        self.path = None
        

    def has_path(self):
        return self.path is not None or len(self.frame.timestep) > 1
    
    def get_pedestrianId(self):
        return self.id
            
    def get_start(self):
        return self.start
    
    def get_path(self):
        if self.path is None and self.has_path():
            self.path = LineString(list(zip(self.frame.x.values, self.frame.y.values)))
            
        return self.path

def plot_source(sourceId, mapping, path, sources, skip=1, mode='all'):
    pedestrians = mapping[mapping['source'] == sourceId].pedestrianId.values
    plot_trajectories(pedestrians, path, sources, skip=skip, mode=mode)

def stripMillies(row):
        row = row.replace(microsecond=0)
        return row  
        

def load_pedestrians(filename, path, mode='all',time_frame=0):
    # read chuncks of the csv file
    block_size = 10000
    
    block = pd.read_csv(os.path.join(path, filename+'.csv'), sep=',',header=None, skiprows=0, nrows=block_size)
    block.insert(1,'datetime',pd.to_datetime(block[0],unit='ms'))
    
    block['datetime'] = block['datetime'].apply(stripMillies)
    
    #determine dts
    t0 = block['datetime'][0]
    tend = pd.to_datetime(block['datetime'].tail(1).item())
    
    print("t0:",t0, '\ntend:',tend)
    print('\n\n')
    
    block.insert(2,'dt',block[0].diff())
    print(block.head(20))
    
    block[0] = block[0].divide(1000)
    block[0] = block[0].apply(floor)
    
    
    block = block.drop_duplicates(0,keep='first')
    print(block.head(20))
    
    
    
def plot_trajectories(pedestrians, path, sources, skip=1, mode='all', showsources=True):
    minx = -46;
    maxx = 10;
    miny = -7;
    maxy = 8;
    
    pedestrians = pedestrians[::skip]
    
    fig, ax = mpl.subplots(ncols=1, nrows=1, figsize=(11, 6), dpi=96)
    
    for ped in tqdm(pedestrians, total=len(pedestrians)):
        print(pd.read_csv(os.path.join(path, ped+'.csv')))
        pedestrian = Pedestrian(pd.read_csv(os.path.join(path, ped), names=['timestep', 'pedestrianId', 'x', 'y']))
        x, y = pedestrian.get_path().xy
        
        if 'start' in mode:
            ax.plot(x[0], y[0], 'c.', markersize=5, label='start')
        
        if 'end' in mode:
            ax.plot(x[-1], y[-1], 'y.', markersize=5, label='end')
        
        if mode == 'traj':
            ax.plot(x, y, 'k', linewidth=0.2)
        
        
    
    img = mpl.imread("../Layout2.png")
    
    ax.imshow(img, extent=[minx, maxx, miny - 3, maxy + 4])
    
    if showsources:
        colors = ['b', 'g', 'r', 'c', 'm', 'y']
        for index, s in enumerate(sources):
            ax.add_patch(PolygonPatch(s, linewidth=2, edgecolor=colors[index % len(colors)], facecolor='none', label=str(index)))
    
    handles, labels = fig.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    
    ax.legend(by_label.values(), by_label.keys(), loc='lower left', bbox_to_anchor=(0, 1.02, 1, 0.2), ncol=len(sources) + 2)
    
    ax.grid()
    

In [ ]:
base = os.getcwd()
print(base)

data_dir = os.path.join(base,'../test')
print(data_dir)

file_names = ['objectstream_20190905_BS_sensor6_by_timestep']

In [ ]:
mpl.rcParams['figure.figsize'] = [20, 5]
pets = load_pedestrians(file_names[0],data_dir)

In [ ]:
l1 = np.linspace(0,100,1001)
print(len(l1))
dt2 = 0.5
dt1 = 0.1

factor = int(0.5/0.1)
print(factor)
l2 = l1[0::5]
print(l2[-1])
print(len(l2))

1000/5 + 1


In [ ]:
# Generate data on commute times.
size, scale = 1000, 10
commutes = pd.Series(np.random.gamma(scale, size=size) ** 1.5)

commutes.plot.hist(grid=True, bins=20, rwidth=0.9,
                   color='#607c8e')
mpl.title('Commute Times for 1,000 Commuters')
mpl.xlabel('Counts')
mpl.ylabel('Commute Time')
mpl.grid(axis='y', alpha=0.75)